In [1]:
import numpy as np
import cv2

<h2> Global Thresholding </h2>

In [2]:
def global_threshold(image, deltaT = 0):
    
    #calculate the average intensity of the image as starting Threshold
    T = int(np.mean(image))
    Tnew = None
    #for storing history
    cache = [abs(T)]

    #loop till T - Tnew > deltaT
    while not Tnew or abs(T - Tnew) > deltaT :
        
        if Tnew is not None:
            T = Tnew
        #some parameters we will need
        m1 = 0    
        m2 = 0
        total1 = 0
        total2 = 0
        
        #threshold
        for pixel in image.reshape(-1):
            
            if pixel > T:
                #incorporate sum in m1
                m1 += pixel
                total1 += 1
                continue
                
            m2 += pixel
            total2 += 1
        
        m1 /= total1
        m2 /= total2
        
        Tnew = int((m1 + m2) / 2)
        
        cache.append(int(Tnew))
        
    
    #threshold based on the final T value
    new_img = []
    
    for pixel in image.reshape(-1):
        
        if pixel > Tnew:
            new_img.append(255)
            continue
            
        new_img.append(0)
        
    
    new_img = np.array(new_img, dtype = np.uint8).reshape(image.shape)
    
    return (new_img, Tnew, cache)
            

<h2> Otsu's Method </h2>

In [3]:
def otsu_threshold(image) :
    '''
    This function calculates the optimal global threshold using otsus method.
    Inputs:
    image => a numpy array respresentation of the grayscale image
    
    returns:
    k_star => optimal global threshold
    cache :{
    P1_kstar => probability that pixel is in class 1
    P2_kstar => probability that pixel is in class 2
    n_star => separibility measure constant. (more means highly separable classes)
    sigma_g => global variance
    m => list of all cumulative means
    P1 => list of all cumulative sums
    sigma_b => list of all between-cluster variances
    
    }
    
    '''
    totalPixels = image.reshape(-1).shape[0]
    image_dup = np.copy(image).reshape(-1)
    intensities = np.array(range(0, 256))
    
    #get the histogram frequencies
    hist, _ = np.histogram(image, bins = 256, range = (0, 255))
    #calculate probabilities
    probs = hist / totalPixels
    
    #stores cumulative probability sum
    P1 = []
    
    #stores cumulative mean | expected value
    m = []
    
    #store between class variance
    sigma_B = []
    
    #find global expected intensity 
    m_G = np.sum(probs[0 : 256] * intensities[0 : 256])
            
    #loop through all possible k values
    for k in range(0, 256):
        
        #append cumulative probability
        P1.append(np.sum(probs[0 : k + 1]))
        
        #append cumulative mean
        m.append(np.sum(probs[0 : k + 1] * intensities[0 : k + 1]))
        
        #append value of between-class variance
        if P1[k] * (1 - P1[k]) > 0:
            variance = np.power((m_G * P1[k] - m[k]), 2) / (P1[k] * (1 - P1[k]))
            
        else:
            variance = 0
            
        sigma_B.append(variance)
        
    
    #get index where sigma_B is maximum
    sigma_B = np.array(sigma_B)
    max_indices = np.where(sigma_B == sigma_B.max())[0]
    
    #average the max indices
    k_star = int(np.mean(max_indices))
    
    #calculate global variance
    sigma_G = np.sum(np.power(intensities - m_G, 2) * probs)
    
    #calculate separability measure
    n_star = sigma_B[k_star] / sigma_G
    
    #calculate P1_kstar
    P1_kstar = P1[k_star]
    
    #calculate P2 kstar
    P2_kstar = 1 - P1_kstar
    
    cache = {
        "n_star" : n_star,
        "P2_kstar" : P2_kstar,
        "P1_kstar" : P1_kstar,
        "sigma_g" : sigma_G,
        "P1" : P1,
        "m" : m,
        "sigma_b" : sigma_B
        
    }
    
    return (k_star, cache)